<a href="https://colab.research.google.com/github/estefaniabrito/SYS/blob/main/Ejerciciostransformadasz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#UNIVERSIDAD NACIONAL DE COLOMBIA
Ejercicios transformada Z

Estefania Ramirez Brito


In [ ]:
#! pip install youtube-dl
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz (2.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for yt-dlp: filename=yt_dlp-2025.2.19-py3-none-any.whl size=2947654 sha256=5ec7632f415bf474a1735d7fbfe5113fba2fb18f08231ed598bc522f6ce9a5ad
  Stored in directory: /tmp/pip-ephem-wheel-cache-k85m_g74/wheels/2d/79/97/7209650ef73114e0fe0603480da012ad3afacb9cae6b8acd9a
Successfully built yt-dlp
  Attempting uninstall: yt-dlp
    Found existing installation: yt-dlp 2025.2.19
    Uninstalling yt-dlp-2025.2.19:
      Successfully uninstalled yt-dlp-2025.2.19


In [ ]:
link="https://www.youtube.com/watch?v=pwQXXNo2Hxo&list=RDpwQXXNo2Hxo&start_radio=1"
!yt-dlp --extract-audio -o "audio" --audio-format mp3 {link}
!yt-dlp -f bestaudio "https://www.youtube.com/watch?v=pwQXXNo2Hxo"


[youtube] Extracting URL: https://www.youtube.com/watch?v=pwQXXNo2Hxo
[youtube] pwQXXNo2Hxo: Downloading webpage
[youtube] pwQXXNo2Hxo: Downloading tv client config
[youtube] pwQXXNo2Hxo: Downloading player f6e09c70
[youtube] pwQXXNo2Hxo: Downloading tv player API JSON
[youtube] pwQXXNo2Hxo: Downloading ios player API JSON
ERROR: [youtube] pwQXXNo2Hxo: Sign in to confirm you’re not a bot. Use --cookies-from-browser or --cookies for the authentication. See  https://github.com/yt-dlp/yt-dlp/wiki/FAQ#how-do-i-pass-cookies-to-yt-dlp  for how to manually pass cookies. Also see  https://github.com/yt-dlp/yt-dlp/wiki/Extractors#exporting-youtube-cookies  for tips on effectively exporting YouTube cookies
[youtube] Extracting URL: https://www.youtube.com/watch?v=pwQXXNo2Hxo
[youtube] pwQXXNo2Hxo: Downloading webpage
[youtube] pwQXXNo2Hxo: Downloading tv client config
[youtube] pwQXXNo2Hxo: Downloading player f6e09c70
[youtube] pwQXXNo2Hxo: Downloading tv player API JSON
[youtube] pwQXXNo2Hxo: D

In [ ]:
!ffmpeg -i audio.mp3 output.wav
!pip install soundfile

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
import soundfile as sf # para instalar pip install soundfile
#lee archivos wav
nombre_out = "output.wav"
x, fs = sf.read(nombre_out)

# read speech signal from file
print('Frecuencia de muestreo %.2f[Hz]\naudio %s' % (fs,nombre_out))

LibsndfileError: Error opening 'output.wav': System error.

In [ ]:
x.shape[0]/fs #segundos de la canción

In [ ]:
xpro = x.copy() #copiar archivos para procesar
ti = 30
tf = 60
xs = xpro[int(ti*fs):int((tf*fs)),:]

In [ ]:
#Para escuchar el trozo de audio con el que se va a trabajar:
#No usar un trozo muy grande o el entorno se puede desconectar
from IPython.display import Audio
Audio([xs[:,1], xs[:,0]],rate=fs)

#Filtro Butterworth


In [ ]:
from scipy.signal import butter as bw
from scipy.signal import freqz_zpk

In [ ]:
import numpy as np
from scipy.signal import butter, freqz
from matplotlib import pyplot as plt
from matplotlib.patches import Circle

# Función para el diseño del filtro
def get_filter_params():
    N = int(input("Ingrese el orden del filtro (entero positivo): "))
    filt_type = input("Ingrese el tipo de filtro (lowpass, highpass, bandpass, bandstop): ").strip().lower()

    # Si el filtro es pasa-banda o rechaza-banda, necesitamos 2 frecuencias de corte
    if filt_type in ['bandpass', 'bandstop']:
        Wn = list(map(float, input("Ingrese las frecuencias de corte como dos valores separados por coma (Hz): ").split(',')))
        if len(Wn) != 2:
            raise ValueError("Debe ingresar exactamente dos frecuencias de corte.")
    else:
        Wn = float(input("Ingrese la frecuencia de corte (Hz): "))

    fs = float(input("Ingrese la frecuencia de muestreo (Hz): "))
    return N, Wn, filt_type, fs

# Función para diseñar el filtro
def design_filter(N, Wn, btype, fs):
    # Convertir las frecuencias de corte a frecuencias normalizadas
    if isinstance(Wn, list):  # Para bandpass o bandstop
        Wn = [w / (0.5 * fs) for w in Wn]
    else:  # Para lowpass o highpass
        Wn = Wn / (0.5 * fs)

    # Diseño del filtro usando butter
    b, a = butter(N, Wn, btype=btype, output='ba')
    return b, a

# Función para mostrar la respuesta en frecuencia
def plot_freq_response(filter_name, w, h, N):
    fig = plt.figure()
    ax1 = fig.add_subplot(1, 1, 1)
    ax1.set_title(filter_name + ' Digital filter frequency response, order=' + str(N))
    ax1.plot(w, 20 * np.log10(abs(h)), 'b')
    ax1.set_ylabel('Amplitude [dB]', color='b')
    ax1.set_xlabel('Frequency [Hz]')
    ax1.grid()
    ax2 = ax1.twinx()
    angles = np.unwrap(np.angle(h))
    ax2.plot(w, angles, 'g')
    ax2.set_ylabel('Angle [radians]', color='g')
    plt.axis('tight')
    plt.show()

# Función para mostrar los polos y ceros
def show_zp(b, a, title='Z-plane'):
    # Cálculo de los polos y ceros
    zeros, poles, gain = np.roots(b), np.roots(a), np.polyval(a, 0)
    ax = plt.gca()

    ax.plot(np.real(zeros), np.imag(zeros), 'bo', fillstyle='none', ms=10)
    ax.plot(np.real(poles), np.imag(poles), 'rx', fillstyle='none', ms=10)
    unit_circle = Circle((0, 0), radius=1, fill=False, color='black', ls='solid', alpha=0.9)
    ax.add_patch(unit_circle)
    ax.axvline(0, color='0.7')
    ax.axhline(0, color='0.7')

    plt.title(title)
    plt.xlabel(r'Re{$z$}')
    plt.ylabel(r'Im{$z$}')
    plt.axis('equal')
    plt.xlim((-2, 2))
    plt.ylim((-2, 2))
    plt.grid()

# Main: Solicitar parámetros y diseñar el filtro
def main():
    # Obtener parámetros del filtro
    N, Wn, filt_type, fs = get_filter_params()

    # Diseñar el filtro
    b, a = design_filter(N, Wn, filt_type, fs)

    # Mostrar respuesta en frecuencia
    w, h = freqz(b, a, worN=8000, fs=fs)
    plot_freq_response(filt_type, w, h, N)

    # Mostrar polos y ceros
    show_zp(b, a)

# Ejecutar el programa
if __name__ == "__main__":
    main()


##Chevyseb 1